# 召回框架

节选自：[万变不离其宗：用统一框架理解向量化召回](https://zhuanlan.zhihu.com/p/345378441)

召回算法品类众多且形态迥异，看似很难找出共同点。现在比较流行的召回算法，比如 item2vec、DeepWalk、Youtube 召回算法、Airbnb 召回算法、双塔模型、Facebook 的 EBR 算法 ...

* 从召回方式上分，有的直接给 user 找他喜欢的 item（u2i）；有的是拿用户喜欢的 item 找相似 item（i2i）
* 从算法实现上分，有的来自“前深度学习”时代；有的基于深度学习；有的基于图算法
* 从优化目标上分，有的属于一个超大规模的多分类问题，优化 softmax loss；有的基于 Learning-To-Rank，优化的是 hinge loss

然而，以上这些召回算法，其实都可以被一个统一的框架所囊括。即我们主要关注召回算法的 Near、Far、Embedding、Loss 即可。

## Near：如何定义近

一般取决于不同的召回方式：

* i2i 召回：我们认为同一个用户在同一个 session 交互过的 item 在向量空间中是相近的
* u2i 召回：一个用户与其交互过的 item 应该是相近的

## Far：如何定义远

其实就是举反例。举出的 $<x,y_{-}>$ 反例，要能够让模型见识到形形色色、五花八门的 $x,y_{-}$ 之间的差异性，达到让模型`开眼界、见世面`的目的。特别是在训练 u2i 召回模型时，一个非常重要的原则是，`千万不能只拿“曝光未点击”做负样本`，否则，正负样本都来自于“曝光样本”，都是与 user 比较匹配的 item，而在上百万的候选 item 中，绝大部分 item 都是与 user 兴趣八竿子打不着的。这种训练数据和预测数据之间的 bias，将导致召回模型上线后水土不服。

为了达到以上目的，获得 $y_{-}$ 最重要的方式就是在整个 $y$ 的候选集中随机采样：

* 随机采样得到的 $y_{-}$ 也可能与 $x$ 是相近的，但是这种概率一般来说是很小的，可忽略。
* 随机采样并非按频次 sample，那样会导致热门 item 霸占训练集，从而失去开眼界、见世面的目的。因此在采样时，需要打压热门 item，比较有效的一种方式是学习 word2vec 中打压高频词的方法。

但是，只随机采样获得 $y_{-}$ 也有问题，就是会导致模型的精度不足。因此，不同的算法，通常还会采取不同的方式获得 `hard negatives`

## Embedding：如何生成向量

u2i 召回，与排序，虽然都是建模 user 和 item 的匹配关系，但是在样本、特征、模型上都有显著不同。最显著的差别，简言之就是：`排序鼓励交叉、召回要求解耦`。

# Loss：如何优化

在召回中，我们的目的是区分出正样本和形形色色的负样本（大多是随机采样的）。因此，召回算法往往采用 pairwise LearningToRank，建模顺序的相对准确性

* 样本往往是 $<user, item_{+}, item_{-}>$ 的三元组形式。优化目标是 $<user, item_{+}>$ 的匹配程度要远高于 $<user, item_{-}>$ 的匹配程度

为了实现以上目标，有几种 Loss 可供选择

一种是 sampled softmax loss，这时 $user$ 选中 $item+$ 的概率是

$$\frac{\exp{(v_{u}\cdot v_{i+}})}{\sum_{I}\exp{(v_{u}\cdot v_{i}})}$$

但是由于计算时牵扯到整个候选 item 的集合 $|I|$，计算量太大。所以实际优化的是 sampled softmax loss

另一种是 margin hinge loss，我们期望 $<user, item_{+}>$ 的匹配程度要比 $<user, item_{-}>$ 的匹配程度高出一个阈值

$$loss = \max(0, margin - user\cdot item_{+} + user\cdot item_{-})$$